In [125]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse
from tqdm.notebook import tqdm

In [3]:
def get_event_judges(title, date, place, online_table_link):
    if online_table_link is None:
        return None
    df = pd.DataFrame(columns=['title','date','place','online_link','category', 'segment', 'position', 'name'])
    page = requests.get(online_table_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    all_off = soup.find_all('a', {'href': re.compile(r'SEG[0-9]{3}OF')})
    qty = 0
    for link in all_off:
        off_link = urllib.parse.urljoin(online_table_link, link.get('href'))
        page = requests.get(off_link)
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'html.parser')
            if len(soup.find_all('h2')) < 2:
                continue
            category, segment = [x.strip() for x in soup.find_all('h2')[1].text.split('-')[:2]]
            lines = soup.find_all('tr')
            for line in lines[1:]:
                cells = line.find_all('td')
                if cells[0].text != '\xa0':
                    df.loc[len(df)] = [title, date, place, off_link, category, segment, cells[0].text, cells[1].text]
    return df

In [152]:
def get_all_links(online_table_link):
    if online_table_link is None:
        return None
    page = requests.get(online_table_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    all_res = soup.find_all('a', {'href': re.compile(r'SEG[0-9]{3}.HTM')})
    return list(map(lambda x: urllib.parse.urljoin(online_table_link, x.get('href')), all_res))

In [153]:
def parse_result_line(line, data):
    cells = line.find_all('td')
    if len(cells) < 10:
        return False
    rank = cells[0].text
    full_name = cells[1].a.text
    club = cells[1].text[len(full_name):]
    full_name = full_name.replace("ё", "е").replace("Ё", "Е")
    club = club.replace("ё", "е").replace("Ё", "Е")
    full_name = [x.strip() for x in full_name.split()]
    if len(full_name) == 2:
        first_name = full_name[0]
        middle_name = ""
        last_name = full_name[1]
    elif len(full_name) == 3:
        first_name = full_name[0]
        middle_name = full_name[1]
        last_name = full_name[2]
    else:
        return False
    tss = float(cells[2].text)
    tes = float(cells[3].text)
    pcs = float(cells[5].text)
    data['rank'].append(rank)
    data['firstname'].append(first_name)
    data['middlename'].append(middle_name)
    data['lastname'].append(last_name)
    data['club'].append(club)
    data['tss'].append(tss)
    data['tes'].append(tes)
    data['pcs'].append(pcs)
    return True

In [154]:
def get_category_and_segment(page):
    soup = BeautifulSoup(page.text, 'html.parser')
    if len(soup.find_all('h2')) >= 1:
        return  [x.strip() for x in soup.find_all('h2')[1].text.split(' - ')[:2]]

In [156]:
def parse_results(entries_link, data, info):
    if entries_link is None:
        return None
    page = requests.get(entries_link)
    if page.status_code != 200:
        return None
    category, segment = get_category_and_segment(page)
    soup = BeautifulSoup(page.text, 'html.parser')
    lines = soup.find_all('tr')
    for line in lines:
        if parse_result_line(line, data):
            data['category'].append(category)
            data['segment'].append(segment)
            for key in info:
                data[key].append(info[key])

In [157]:
def parse_event(event_link, data):
    soup = BeautifulSoup(requests.get(event_link).text, 'html.parser')
    title = soup.find("h1", "entry-title").text
    date = soup.find("div", "competition-date").p.text.split('-')[0].strip()
    place = soup.find("div", "competition-place").p.span.text.strip()
    online = soup.find("div", "competition-file")
    if online is not None:
        online_link = online.a.get('href')
        online_link = online_link if online_link[-1] == '/' else online_link + '/'
        segments = get_all_links(online_link)
        for segment in segments:
            parse_results(segment, data, {'date': date, 'place': place, 'online': online_link})

In [158]:
def parse_season(season_link, data):
    page = requests.get(season_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    competitions = soup.find_all('tr')
    for i in tqdm(range(len(competitions))): 
        title = competitions[i].find_all('td')[1]
        if title.a is not None:
            online_link = None
            proto_link = None
            event_link = title.a.get('href')
            parse_event(event_link, data)

In [159]:
data = {
    'date': [],
    'place': [], 
    'online':[],
    'category': [],
    'segment': [],
    'rank': [],
    'firstname': [],
    'middlename': [],
    'lastname': [],
    'club': [],
    'tss': [],
    'tes': [],
    'pcs': []
}
for season in range(2024, 2025):
    parse_season(f"http://ffkkmo.ru/calendar/?season={season}", data)
df = pd.DataFrame.from_dict(data)
df

  0%|          | 0/61 [00:00<?, ?it/s]

,date,place,online,category,segment,rank,firstname,middlename,lastname,club,tss,tes,pcs
0,14.09.2024,"г. Клин, ул. Карла Маркса, д. 99, Ледовый двор...",http://ffkkmo.ru/events/2425/klin09/,"3 юношeский рaзряд, дeвочки",Произвольная программа,1,Александра,,МАСЛОВА,"г.о. Химки, МАУ ДО ""СШ по ЗВС""",15.86,5.99,9.87
1,14.09.2024,"г. Клин, ул. Карла Маркса, д. 99, Ледовый двор...",http://ffkkmo.ru/events/2425/klin09/,"3 юношeский рaзряд, дeвочки",Произвольная программа,2,Мария,,ПАВЛЕНКО,"г.о. Химки, МАУ ДО ""СШ по ЗВС""",15.65,5.39,10.26
2,14.09.2024,"г. Клин, ул. Карла Маркса, д. 99, Ледовый двор...",http://ffkkmo.ru/events/2425/klin09/,"3 юношeский рaзряд, дeвочки",Произвольная программа,3,Юмабика,,КИРГИЗБАЕВА,"г. Клин, МБУ ДО СШОР ""Клин спортивный""",14.76,4.88,9.88
3,14.09.2024,"г. Клин, ул. Карла Маркса, д. 99, Ледовый двор...",http://ffkkmo.ru/events/2425/klin09/,"3 юношeский рaзряд, дeвочки",Произвольная программа,4,Нина,,ЯКУБОВИЧ,"г. Долгопрудный, АУ ФСК ""Салют""",13.09,4.10,9.99
4,14.09.2024,"г. Клин, ул. Карла Маркса, д. 99, Ледовый двор...",http://ffkkmo.ru/events/2425/klin09/,"3 юношeский рaзряд, дeвочки",Произвольная программа,5,Ольга,,СТУПАКОВА,"г. Клин, МБУ ДО СШОР ""Клин спортивный""",12.56,4.43,8.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5439,24.05.2025,"г. Электросталь, ул. Радио, д. 3, ЛДС ""Кристалл""",http://ffkkmo.ru/events/2425/el05/,"Юный фигуриcт, дeвoчки",Произвольная программа,1,Вероника,Ивановна,ПЕТРОВА,"г. о. Лыткарино, МАУ ДО ""СШ Лыткарино""",13.77,6.27,7.50
5440,24.05.2025,"г. Электросталь, ул. Радио, д. 3, ЛДС ""Кристалл""",http://ffkkmo.ru/events/2425/el05/,"Юный фигуриcт, дeвoчки",Произвольная программа,2,София,Алексеевна,АШАРИНА,"г. о. Электросталь, МБУДО «СШОР «Кристалл - Во...",13.29,5.69,7.60
5441,24.05.2025,"г. Электросталь, ул. Радио, д. 3, ЛДС ""Кристалл""",http://ffkkmo.ru/events/2425/el05/,"Юный фигуриcт, дeвoчки",Произвольная программа,3,Лусинэ,Сергеевна,ТЕПОЯН,"г. о. Балашиха, МАУ ДО ""СШОР имени Ю. Е. Ляпкина""",11.75,5.75,6.00
5442,24.05.2025,"г. Электросталь, ул. Радио, д. 3, ЛДС ""Кристалл""",http://ffkkmo.ru/events/2425/el05/,"Юный фигуриcт, дeвoчки",Произвольная программа,4,Екатерина,Дмитриевна,ИВАНОВА,"г. о. Электросталь, МБУДО «СШОР «Кристалл - Во...",11.33,5.53,6.30


In [146]:
df[(df['LastName'] == 'ИВАНОВА') & (df['FirstName'] == 'Ульяна')]

,Date,Place,Online,Category,Segment,Rank,FirstName,MiddleName,LastName,Club,TSS,TES,PCS
975,13.11.2024,"г.о. Наро-Фоминск, г. Селятино, Спорткомбинат ...","[[[], Онлайн табло]]","3-й cпopтивный, дeвушки",Произвольная программа,5,Ульяна,,ИВАНОВА,"МАУ ""ГС Авангард"", г.о. Домодедово",34.72,13.71,21.01
1816,18.12.2024,"г. Ногинск, ул. Климова, д. 48, к. А, ФОК ""Лед...","[[[], Онлайн табло]]","3 cпoртивный рaзряд, девушки",Произвольная программа,8,Ульяна,,ИВАНОВА,"г.о. Домодедово, МАУ ""ГС Авангард""",33.57,15.43,18.14
2062,28.12.2024,"г. Сергиев Посад, Новоугличское ш., д.73 В, Ле...","[[[], Онлайн табло]]","3 cпoртивный рaзряд, дeвушки",Произвольная программа,9,Ульяна,,ИВАНОВА,"г.Домодедово, МАУ «ГС «Авангард»",31.62,12.58,19.04
2182,23.01.2025,"г. Ногинск, ул. Климова, 48, кор. А,ФОК Ледова...","[[[], Онлайн табло]]","3 спортивный разряд, девушки",Произвольная программа,8,Ульяна,,ИВАНОВА,"г.о. Домодедово, МАУ ""ГС Авангард""",33.03,12.19,20.84
2358,05.02.2025,"г. Химки, ул. Мичурина с.24 к.1, Ледовая арена","[[[], Онлайн табло]]","3 cпoртивный рaзряд, дeвушки",Произвольная программа,6,Ульяна,,ИВАНОВА,"г.о. Домодедово, МАУ ГС Авангард",33.60,15.46,18.14
2832,18.02.2025,"г. Ногинск, ул. Климова, д. 48, к. А, ФОК ""Лед...","[[[], Онлайн табло]]","3 cпоpтивный разряд, девушки",Произвольная программа,7,Ульяна,,ИВАНОВА,"МАУ ""ГС ""Авангард"" г.о.Домодедово",35.03,15.54,19.49
3091,04.03.2025,"г. Селятино, Спорткомбинат ""Строитель"" Ледовая...","[[[], Онлайн табло]]","3 cпoртивный рaзряд, дeвушки",Произвольная программа,6,Ульяна,,ИВАНОВА,"г. Домодедово, МАУ ГС Авангард",36.53,15.10,21.43
3314,18.03.2025,"г. Щелково, ул. Фабричная, д. 4, ЛД им. В. Тре...","[[[], Онлайн табло]]","3 cпортивный pазpяд, дeвoчки",Произвольная программа,13,Ульяна,,ИВАНОВА,"г.о. Домодедово, МАУ ""ГС Авангард""",34.22,15.32,18.90
3839,03.04.2025,"г. Ногинск, ул. Климова, д. 48, к. А, ФОК ""Лед...","[[[], Онлайн табло]]","3 cпoртивный рaзряд, дeвушки",Произвольная программа,6,Ульяна,,ИВАНОВА,"г.Домодедово, МАУ «ГС «Авангард»",36.79,15.02,21.77
4319,19.04.2025,"г. Щелково, ул. Фабричная д.4, Ледовая арена и...","[[[], Онлайн табло]]","3 cпoртивный рaзряд, дeвушки",Произвольная программа,9,Ульяна,,ИВАНОВА,"г. Домодедово, МАУ «ГС «Авангард»",32.86,14.86,18.00


In [5]:
df_sportsmens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25980 entries, 0 to 123
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        25980 non-null  object
 1   date         25980 non-null  object
 2   place        25980 non-null  object
 3   online_link  25980 non-null  object
 4   category     25980 non-null  object
 5   rank         25980 non-null  object
 6   name         25980 non-null  object
 7   club         25980 non-null  object
 8   points       25980 non-null  object
dtypes: object(9)
memory usage: 2.0+ MB


In [6]:
df_judges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24705 entries, 0 to 170
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        24705 non-null  object
 1   date         24705 non-null  object
 2   place        24705 non-null  object
 3   online_link  24705 non-null  object
 4   category     24705 non-null  object
 5   segment      24705 non-null  object
 6   position     24705 non-null  object
 7   name         24705 non-null  object
dtypes: object(8)
memory usage: 1.7+ MB


In [8]:
df_sportsmens.to_csv("ffkkmo_dump.csv")

In [ ]:
df_judges.to_csv("df_judges_dump.csv")

In [160]:
df.to_csv("ffkkmo_2425_dump.csv")